In [27]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.tools import DuckDuckGoSearchRun
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType, Tool, initialize_agent
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import format_tool_to_openai_tool

load_dotenv()

DRAMMEN_OGC_API_URL = 'https://alenos-tester001.azurewebsites.net/collections/kommuner/items?f=json&kommunenummer=3005'

In [39]:
tools = [DuckDuckGoSearchRun()]
oai_tools = [format_tool_to_openai_tool(tool) for tool in tools] + [{"type": "code_interpreter"}]

oai_tools

[{'type': 'function',
  'function': {'name': 'duckduckgo_search',
   'description': 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.',
   'parameters': {'title': 'DDGInput',
    'type': 'object',
    'properties': {'query': {'title': 'Query',
      'description': 'search query to look up',
      'type': 'string'}},
    'required': ['query']}}},
 {'type': 'code_interpreter'}]

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [30]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind(tools=oai_tools)
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=lc_tools, verbose=True)

ThreadMessage(id='msg_bizEJNcQ4PJqpYta3ByS70qE', assistant_id='asst_tt9PzpNPnCrBRKqBNlMUUH1F', content=[MessageContentText(text=Text(annotations=[], value='The result of \\(10 - 4^{2.7}\\) is approximately \\(-32.22\\).'), type='text')], created_at=1702460008, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_pvS3ZrULfUbcv5HyCSwchmIu', thread_id='thread_tp5GPrd7UZzsNno7haVqJWfz')


BadRequestError: Error code: 400 - {'error': {'message': 'Runs in status "completed" do not accept tool outputs.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
agent_executor.invoke({"input": f"{DRAMMEN_OGC_API_URL}\n\nPlot the contents on this API endpoint."})